In [1]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sym
import sklearn as sk
import seaborn as sns

In [2]:
df_test = pd.read_csv("./test_data.csv")
df_train = pd.read_csv("./train_data.csv")
df_test.head()

,file_name,path,class
0,NORMAL18.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Normal
1,DR106.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Diabetic
2,AMRD44.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Age_related_Macular_Degeneration
3,CSR37.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Central_serous
4,NORMAL110.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Normal


In [3]:
df_train.head()

,file_name,path,class
0,AMRD46.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Age_related_Macular_Degeneration
1,CSR57.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Central_serous
2,DR27.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Diabetic
3,CSR9.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Central_serous
4,DR19.jpeg,/Users/asmitganguly/Developer/internships/UWat...,Diabetic


In [4]:
import pandas as pd
from PIL import Image
from io import BytesIO
import base64
from llama_cpp import Llama
from sklearn.metrics import accuracy_score

# Load the model
llm = Llama(model_path="/Users/asmitganguly/Library/Caches/llama.cpp/ggml-org_SmolVLM-256M-Instruct-GGUF_SmolVLM-256M-Instruct-Q8_0.gguf", n_ctx=2048)

def image_to_base64_data_uri(file_path):
    with open(file_path, "rb") as img_file:
        base64_data = base64.b64encode(img_file.read()).decode('utf-8')
        return f"data:image/png;base64,{base64_data}"

def construct_prompt(image_data_uri, classes):
    class_list = ', '.join(classes)
    prompt = (
        f"Classify the following image into one of the following classes: {class_list}.\n"
        f"![image]({image_data_uri})"
    )
    return prompt

def encode_image(image_path):
    with Image.open(image_path) as img:
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode()

def classify_image(image_path, classes):
    image_data_uri = image_to_base64_data_uri(image_path)
    prompt = construct_prompt(image_data_uri, classes)
    output = llm.create_completion(prompt=prompt)
    return output['choices'][0]['text'].strip()


llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 49 key-value pairs and 273 tensors from /Users/asmitganguly/Library/Caches/llama.cpp/ggml-org_SmolVLM-256M-Instruct-GGUF_SmolVLM-256M-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = SmolVLM 256M Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = SmolVLM
llama_model_loader: - kv   5:                         general.size_label str         

In [5]:
classes = df_test['class'].unique().tolist()
classes

['Normal',
 'Diabetic',
 'Age_related_Macular_Degeneration',
 'Central_serous',
 'Macular_Hole']

In [6]:
from tqdm import tqdm
# Initialize liclassessts to store predictions and true labels
predictions = []
true_labels = []

for index, row in tqdm(df_test.iterrows()):
    image_path = row['path']
    true_label = row['class']
    predicted_label = classify_image(image_path, classes)
    predictions.append(predicted_label)
    true_labels.append(true_label)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

0it [00:00, ?it/s]

0it [00:00, ?it/s]


ValueError: Requested tokens (87125) exceed context window of 2048